Annie Taylor
1.4.2023

# Download and Reformat iNaturalist Data

#### Import packages and change arcpy defaults

In [15]:
from pyinaturalist import *
import pandas as pd
import arcpy
arcpy.env.overwriteOutput = True

#### Import observations from inat API
Change all four parameters each time you run it for a different species; a few examples are included below. 
The code parameter is the first two letters of each word in the plant's scientific name.
Find the iNaturalist taxon id on wikipedia or https://www.inaturalist.org/taxa/47126-Plantae


In [16]:
name = 'blackoak'
code = 'QUKE'
species = 'Quercus kelloggii'
taxon = 49919

# name = 'soaproot'
# code = 'CHPO'
# species = 'Chlorogalum pomeridianum'
# taxon = 47597

# name = 'blackberry'
# code = 'RUUR'
# species = 'Rubus ursinus'
# taxon = 53445

# Specify criteria for observations
# https://pyinaturalist.readthedocs.io/en/latest/modules/pyinaturalist.v1.observations.html#pyinaturalist.v1.observations.get_observations
# my area = is roughly the AMLT stewardship area, change the four coordinate bounds to bound your study area
response = get_observations(taxon_id=taxon, swlat = 36.077540, swlng = -122.605514, nelat = 37.381013, nelng = -120.292403, page='all', per_page=200, quality_grade='research')

#### View the results in a pandas dataframe

In [51]:
# Select results from JSON Data, and normalize them
# This flattens the nested iNat data structure and converts to pandas dataframe
data = response['results']
df = pd.json_normalize(data)

# Split the location data into latitude and Longitude columns
# this is the format of location col [38.8418469038, -123.0608224869]
df['lat'] = df['location'].apply(lambda x:x[0])
df['lon'] = df['location'].apply(lambda x:x[1])

# Subset dataframe to include only the fields of interest
df = df[['lat','lon','quality_grade','time_observed_at','taxon.name','taxon.preferred_common_name','positional_accuracy','public_positional_accuracy']]
df = df.rename({'taxon.name': 'species_name'}, axis=1)

# Show the first 8 lines of the dataframe below cell
df.head(8)

,lat,lon,quality_grade,time_observed_at,species_name,taxon.preferred_common_name,positional_accuracy,public_positional_accuracy
0,37.176597,-121.519333,research,None,Quercus kelloggii,California black oak,NaN,NaN
1,36.364000,-121.561667,research,2013-06-17T10:39:29-07:00,Quercus kelloggii,California black oak,NaN,NaN
2,37.291085,-122.057652,research,2014-03-07T15:18:52-08:00,Quercus kelloggii,California black oak,5.0,28417.0
3,37.171149,-121.496220,research,None,Quercus kelloggii,California black oak,80.0,80.0
4,37.245260,-121.727447,research,None,Quercus kelloggii,California black oak,48.0,10000.0
5,37.290580,-122.154708,research,2015-03-14T08:49:27-07:00,Quercus kelloggii,California black oak,NaN,NaN
6,36.358546,-121.806450,research,2015-08-26T15:41:00-07:00,Quercus kelloggii,California black oak,106.0,106.0
7,37.274333,-122.139130,research,None,Quercus kelloggii,California black oak,538.0,538.0


#### Define a function to convert the dataframe into a feature class for use in ArcGIS Pro

In [52]:
# This is a user-defined function to push dataframe into arcgis feature class
def addData(fc, occurrences):
    arcpy.management.DeleteRows(fc)
    cursor = arcpy.da.InsertCursor(fc, ["SHAPE@XY","lat","lon","quality_grade", "time_observed_at","species_name","positional_accuracy","public_positional_accuracy"])

    for index, row in occurrences.iterrows():
        xy = (row['lon'],row['lat'])
        cursor.insertRow([xy,row['lat'], row['lon'], row['quality_grade'], row['time_observed_at'], row['species_name'], row['positional_accuracy'],row['public_positional_accuracy']])
    del cursor

#### Create a feature class with the correct fields to hold the iNat data

In [53]:
# Define your workspace and geodatabase
workspace = "D:/1_AMLT/2_Ethnobotanical_Data_Pro/iNaturalist/"

# If file geodatabase already exists, name of file geodatabase
filegdbname = "iNat.gdb"
filegdb = f'{workspace}{filegdbname}'

# # if file geodatabase does not exist, creates file geodatabase
# if (arcpy.Exists(filegdb)) == False:
#     arcpy.management.CreateFileGDB(workspace, filegdbname[:-4], "CURRENT")

# Create a feature class within the geodatabase to add the iNat data to
fc = name # ie "blackoak"
fcPath = f'{filegdb}/{fc}' 

# If feature class does not exist, create feature class
if (arcpy.Exists(fcPath)) == False:
    arcpy.management.CreateFeatureclass(filegdb, fc, "POINT", None, "DISABLED", "DISABLED", 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision', '', 0, 0, 0, '')

    # Fields to create in feature class
    fields = ("lat DOUBLE # # # #;" +
        "lon DOUBLE # # # #;" +
        "quality_grade TEXT # 255 # #;" +
        "time_observed_at TEXT # 255 # #;" +
        "species_name TEXT # 255 # #;" +
        "positional_accuracy DOUBLE # # # #;" +
        "public_positional_accuracy DOUBLE # # # #")

    arcpy.management.AddFields(name, fields) 


#### Add iNat data from the dataframe to the newly created feature class

In [55]:
addData(fcPath, df)

### Processing iNat data for use in SAHM

#### Project and clip iNat data

In [57]:
print(name)
# Project to my SDM projection and clip to study area
output = "D:/1_AMLT/1_SDM/SDM_Pro/SDM.gdb/iNat_" + code
area = "D:/1_AMLT/1_SDM/SDM_Pro/SDM.gdb/BoundingBox" 
outsr = arcpy.Describe(area).spatialReference

arcpy.management.Project(fcPath, output, outsr)

# # Clip to study area boundary if necessary
# inat = output + "_SDM"
# arcpy.analysis.Clip(output, area, inat)

# print number of records/points
result = arcpy.GetCount_management(output)
count = int(result.getOutput(0))
print('Total features in iNat database = ', count)

blackoak
Total features in iNat database =  517


#### Get any points from my ethnobotanical data
Note: this will not work on your computer as it is a confidential dataset.
You may adapt this block to any private/local location datasets that you may have. 

In [59]:
print(name)
# this layer has to be in the active map
# select from the layer by species name
ethnobot = "All_EthnobotData_Merged_012521"

search = "Species LIKE '%" + species + "%'"
print(search)

arcpy.management.SelectLayerByAttribute(ethnobot, "NEW_SELECTION", search)

# export data
gdb = "D:/1_AMLT/1_SDM/SDM_Pro/SDM.gdb/"
out_name = "My_" + code + "_forSAHM"

print('Creating ' + out_name)
arcpy.conversion.FeatureClassToFeatureClass(ethnobot, gdb, out_name)

# clear selection
arcpy.management.SelectLayerByAttribute(ethnobot, "CLEAR_SELECTION")

# print number of records/points
result = arcpy.GetCount_management(out_name)
count = int(result.getOutput(0))
print('Total features in my database = ', count)

blackoak
Species LIKE '%Quercus kelloggii%'
Creating My_QUKE_forSAHM
Total features in my database =  0


#### Merge iNat and personal data
And print the total number of location points

In [60]:
print(name)
# merge the two into one in SDM.gdb, saved as 'All_****_forSAHM'
inat = "D:/1_AMLT/1_SDM/SDM_Pro/SDM.gdb/iNat_" + code
ethno = "D:/1_AMLT/1_SDM/SDM_Pro/SDM.gdb/My_" + code + "_forSAHM"
allpoints = "D:/1_AMLT/1_SDM/SDM_Pro/SDM.gdb/All_" + code + "_forSAHM"

print('Merging inat and ethnobot db into one output')
arcpy.management.Merge([inat, ethno], allpoints)

# print number of records/points
result = arcpy.GetCount_management(allpoints)
count = int(result.getOutput(0))
print('Total features in combined layer = ', count)
# this fc is ready to use, already in WGS84 (best for SAHM)

blackoak
Merging inat and ethnobot db into one output
Total features in combined layer =  517


#### Add a field called 'code' and set to 1
This is necessary for location input data in SAHM

In [61]:
print(name)
# allpoints = "D:/1_AMLT/1_SDM/SDM_Pro/SDM.gdb/All_" + code + "_forSAHM"
# add field named as species code
arcpy.AddField_management(allpoints, code, "LONG")
# set to one
arcpy.management.CalculateField(allpoints, code, 1)
print('Field added')

blackoak
Field added


### Remove location points that fall within the same analysis cell

#### Convert point data to raster
Processing environment = template raster for projection, cell size, snapping, and extent. 
Use code field as value, cell assignment rule = max

In [62]:
print(name)
# Convert these points to a raster
template = r"D:\1_AMLT\1_SDM\SAHM\CovariatesFinal\bio_1_temp.tif"
pointras = r"D:\1_AMLT\1_SDM\SDM_Pro\SDM.gdb\All_" + code + "_forSAHM_Raster"

with arcpy.EnvManager(snapRaster=template, cellSize=template):
    arcpy.conversion.PointToRaster(allpoints, code, pointras, "MAXIMUM","NONE", template, "BUILD")

print('Raster created')

blackoak
Raster created


#### Convert that raster to points
Using centroids, print number of remaining points

In [63]:
print(name)
# convert raster to points (centroids)
finalpoints = r"D:\1_AMLT\1_SDM\SDM_Pro\SDM.gdb\All_" + code + "_forSAHM_FromRaster"

arcpy.conversion.RasterToPoint(pointras, finalpoints)
print('Points from raster created')

# print number of records/points
result = arcpy.GetCount_management(finalpoints)
count = int(result.getOutput(0))
print('Total features in final point layer = ', count)

blackoak
Points from raster created
Total features in final point layer =  228


### Add final fields needed in the locations csv file

#### Add fields for lat as y, long as x in WGS84, responseBinary with 1s, species code column with 1s
Once these SAHM-required fields are added, export the locations as a csv for direct input in your models

In [64]:
print(name)
sppLocation = r"D:\1_AMLT\1_SDM\SAHM\SpeciesLocations"
csvfile = "All_" + code + "_forSAHM_FromRaster.csv"

# add field named as species code
arcpy.AddField_management(finalpoints, code, "LONG")
# set to one
arcpy.management.CalculateField(finalpoints, code, 1)

# add field named responseBinary
arcpy.AddField_management(finalpoints, 'resposeBinary', "LONG")
# set to one
arcpy.management.CalculateField(finalpoints, 'resposeBinary', 1)

# add fields x and y
arcpy.AddField_management(finalpoints, 'x', "DOUBLE")
arcpy.AddField_management(finalpoints, 'y', "DOUBLE")
# calculate lat and long coordinate values in WGS84
arcpy.management.CalculateGeometryAttributes(finalpoints, "x POINT_X;y POINT_Y", '', '', 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "DD")

print('Fields added')

# deleting three unnecessary fields
arcpy.DeleteField_management(finalpoints, ["grid_code", "pointid", "bio_1_temp_1"])

# Export the table to a csv file for input into SAHM
arcpy.TableToTable_conversion(finalpoints, sppLocation, csvfile)
print('Exported to csv')

blackoak
Fields added
Exported to csv
